In [1]:
from pyngrok import ngrok

# Setting an auth token allows us to open multiple
# tunnels at the same time
# ngrok.set_auth_token("2R1tsuod04ayFPLUINN1J0J5qpJ_89tdxKVwtCs9F7kvG1uhv")
# ngrok.set_auth_token("2YGG5TlGcsNjutkY1hua39OM2gv_7XdWGdMHeh6KY4AqMjGiD")
ngrok.set_auth_token("2QWsRU60eUFpOOpFSSkHEqocwAv_2WW7UbufSLbszD94BLGzr")
# <NgrokTunnel: "http://<public_sub1>.ngrok.io" -> "http://localhost:80">
ngrok_tunnel1 = ngrok.connect()
# <NgrokTunnel: "http://<public_sub2>.ngrok.io" -> "http://localhost:8000">

# Start ngrok with the custom configuration file
ngrok_tunnel2 = ngrok.connect(8000)
ngrok_tunnel2

t=2024-04-15T10:37:12+0600 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path="C:\\Users\\ALL USER\\AppData\\Local/ngrok/ngrok.yml" legacy_path="C:\\Users\\ALL USER\\.ngrok2\\ngrok.yml"


<NgrokTunnel: "https://b33d-27-147-206-228.ngrok-free.app" -> "http://localhost:8000">

In [2]:
!lt --port 8000

'lt' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from pyngrok import ngrok

# Setting an auth token allows us to open multiple
# tunnels at the same time
ngrok.set_auth_token("2R1tsuod04ayFPLUINN1J0J5qpJ_89tdxKVwtCs9F7kvG1uhv")

# Start ngrok and specify the port of your local screen sharing server (assuming it's on port 8001)
ngrok_tunnel = ngrok.connect(addr="8001", proto="http", options={"bind_tls": True})

# Print the ngrok public URL
print("Public URL:", ngrok_tunnel.public_url)

In [ ]:
from fastapi import FastAPI, Response
from starlette.responses import StreamingResponse  # Import StreamingResponse from starlette module
import uvicorn
import nest_asyncio
import torch
import time
import json

nest_asyncio.apply()

dahwin = FastAPI()

def generate_tensor(shape):
    # Generate the random tensor
    random_tensor = torch.randn(shape)
    return random_tensor.tolist()  # Convert tensor to list for JSON serialization

@dahwin.post('/stream')
async def stream_data(data: dict):
    shape = data.get('shape', [2, 1, 4])  # Default shape if not provided
    count = data.get('count', 5)  # Default count if not provided
    
    async def event_generator():
        for _ in range(count):
            tensor_data = generate_tensor(shape)
            yield json.dumps(tensor_data) + '\n'
            # time.sleep(2)  # Delay for 2 seconds before sending the next tensor
    
    return StreamingResponse(event_generator(), media_type="text/plain")

if __name__ == '__main__':
    uvicorn.run(dahwin, host='localhost', port=8000)


In [ ]:
import requests
import os
from PIL import Image
from io import BytesIO
import time
# Define the URL of the FastAPI endpoint
url = "http://localhost:8000/video_feed"

# Make a GET request to the streaming endpoint
response = requests.get(url, stream=True)

# Create a directory to store the frames if it doesn't exist
output_dir = "images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize a counter to keep track of the frame number
frame_count = 1
s = time.time()
# Initialize a buffer to accumulate image data
image_buffer = b""

# Iterate over the chunks of data received from the server
for chunk in response.iter_content(chunk_size=1024):
    # Add the current chunk to the buffer
    image_buffer += chunk
    print(chunk)

    # Find the start and end of the image data
    start_idx = image_buffer.find(b'\xff\xd8')
    end_idx = image_buffer.find(b'\xff\xd9')

    # Check if both start and end markers are found
    if start_idx != -1 and end_idx != -1:
        # Extract the image data from the buffer
        image_data = image_buffer[start_idx:end_idx + 2]

        # Create a PIL image from the image data
        image = Image.open(BytesIO(image_data))

        # Save the image as a PNG file
        # image.save(os.path.join(output_dir, f"frame_{frame_count}.png"))
        frame_count += 1

        # Reset the buffer to the remaining data after the current image
        image_buffer = image_buffer[end_idx + 2:]
        if frame_count==1:
            break
print(frame_count)

e = time.time()
l = e-s
print(l)
# Close the response
response.close()


In [3]:
import requests
import os
import numpy as np
import cv2
import time

# Define the URL of the FastAPI endpoint
url = "http://localhost:8000/video_feed"

# Make a GET request to the streaming endpoint
response = requests.get(url, stream=True)

# Create a directory to store the frames if it doesn't exist
output_dir = "images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize a counter to keep track of the frame number
frame_count = 1
start_time = time.time()

# Iterate over the chunks of data received from the server
for chunk in response.iter_content(chunk_size=None):
    # Convert the received chunk into a numpy array
    frame = np.frombuffer(chunk, dtype=np.uint8)
    
    # Decode the JPEG image from the numpy array
    img = cv2.imdecode(frame, cv2.IMREAD_COLOR)
    
    # Save the image as a PNG file
    cv2.imwrite(os.path.join(output_dir, f"frame_{frame_count}.png"), img)
    
    # Increment the frame count
    frame_count += 1
    
    # Break the loop if 200 frames have been processed
    if frame_count == 100:
        break

# Calculate the processing time
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time)

# Close the response
response.close()


Elapsed time: 19.099120140075684


In [17]:
num = 5.53
num1= round(num)
print(num1)
dis = num1-num
print(dis)
if num1>=num and dis>=0.01 and dis<=0.17:
    print(True)

6
0.46999999999999975
